In [62]:
include("../julia/aguirregabiria_core.jl")

compute_fixed_point (generic function with 5 methods)

In [4]:
length_of_price_grid = 10
min_price, max_price = 0.5, 1.5
n_of_lambdas_per_dim = 3
max_iters            = 3
error_tol            = 1e-5

1.0e-5

In [6]:
price_grid = linspace(min_price, max_price, length_of_price_grid)

0.5:0.1111111111111111:1.5

In [8]:
simplex3d = generate_simplex_3dims(n_of_lambdas_per_dim)

6×3 Array{Float64,2}:
 0.0  0.0  1.0
 0.0  0.5  0.5
 0.0  1.0  0.0
 0.5  0.0  0.5
 0.5  0.5  0.0
 1.0  0.0  0.0

## Testing interpolation:

In [63]:
V0 = [2; ones(size(simplex3d,1)-1)]
#tmp_interpV = interpV(simplex3d, simplex3d[:,1])
tmp_interpV = interpV(simplex3d, V0)
methods(tmp_interpV)

# 1 method for generic function "(::interpolate_V)":
(::#interpolate_V#51)(x) in Main at C:\Users\gballari\Documents\GitHub\LearningModels\julia\aguirregabiria_core.jl:40

In [64]:
tmp_V = zeros(size(simplex3d,1))
for i in 1:size(simplex3d,1)
    tmp_V[i] = tmp_interpV(simplex3d[i,:])
end
hcat(V0, tmp_V)

6×2 Array{Float64,2}:
 2.0  2.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0

In [66]:
tmp_V = zeros(11,11)
for (i1, x1) in enumerate(0.0:0.1:1.0)
    for (i2, x2) in enumerate(0.0:0.1:1.0)
        tmp_V[i1,i2] = tmp_interpV([x1, x2])
    end
end
tmp_V

11×11 Array{Float64,2}:
 2.0  1.8  1.6  1.4  1.2  1.0  1.0  1.0  1.0  1.0  1.0
 1.8  1.6  1.4  1.2  1.0  1.0  1.0  1.0  1.0  1.0  0.8
 1.6  1.4  1.2  1.0  1.0  1.0  1.0  1.0  1.0  0.8  0.6
 1.4  1.2  1.0  1.0  1.0  1.0  1.0  1.0  0.8  0.6  0.4
 1.2  1.0  1.0  1.0  1.0  1.0  1.0  0.8  0.6  0.4  0.2
 1.0  1.0  1.0  1.0  1.0  1.0  0.8  0.6  0.4  0.2  0.0
 1.0  1.0  1.0  1.0  1.0  0.8  0.6  0.4  0.2  0.0  0.0
 1.0  1.0  1.0  1.0  0.8  0.6  0.4  0.2  0.0  0.0  0.0
 1.0  1.0  1.0  0.8  0.6  0.4  0.2  0.0  0.0  0.0  0.0
 1.0  1.0  0.8  0.6  0.4  0.2  0.0  0.0  0.0  0.0  0.0
 1.0  0.8  0.6  0.4  0.2  0.0  0.0  0.0  0.0  0.0  0.0

## Test `period_return()` :

In [119]:
include("../julia/aguirregabiria_core.jl")
# lambdas is always a vector of size n_of_lambdas_per_dim
test_lambdas = [0.2, 0.7, 0.1]

3-element Array{Float64,1}:
 0.2
 0.7
 0.1

In [115]:
constant_part = (price_grid - c) * exp(alpha + (sigma_eps^2)/2)

0.0:0.3422463165464479:3.080216848918031

In [116]:
summation = exp.(log.(price_grid)*betas_transition' * test_lambdas)
constant_part .* summation

10-element Array{Float64,1}:
 0.0    
 1.20152
 1.56949
 1.63445
 1.58379
 1.49084
 1.38604
 1.28226
 1.18475
 1.09534

In [117]:
period_return(price_grid, test_lambdas)

10-element Array{Float64,1}:
 0.0    
 1.20152
 1.56949
 1.63445
 1.58379
 1.49084
 1.38604
 1.28226
 1.18475
 1.09534

## Test `dmd_transition_fs()`, `belief()`, `update_lambdas()` :

In [192]:
include("../julia/aguirregabiria_core.jl")
dmd_transition_fs(1, 1.2, 1)

3-element Array{Float64,1}:
 0.438616
 0.526604
 0.611573

In [193]:
belief(1, 1.2, 1, test_lambdas)

0.5175034896003625

In [194]:
update_lambdas(0.3, 2, 1, test_lambdas)

3-element Array{Float64,1}:
 0.0352179
 0.656158 
 0.308624 

## Build `V0` :

In [198]:
optimal_price = myopic_price(test_lambdas)

0.8225806451612903

In [203]:
V_0(x) = period_return(optimal_price, test_lambdas)

V_0 (generic function with 1 method)

## Test `E0fV(Vguess, price_grid, lambda_weights)` :

In [204]:
new_lambdas(x) = update_lambdas(x, price_grid[1], ~, test_lambdas)
new_belief(x)  = belief(x, price_grid[1], ~, test_lambdas)
integrand(x) = V_0(new_lambdas(x)[1:end-1]) * new_belief(x)

integrand (generic function with 1 method)

In [205]:
V_0(test_lambdas)

1.6349875782616206

In [212]:
logd_min, logd_max = -6, 2.3
integrand(1)

0.005090286038995879

In [222]:
A = quadgk(integrand, logd_min, logd_max, maxevals=250)[1]

0.3112581338114793

In [221]:
include("../julia/aguirregabiria_core.jl")
@time A = E0fV(V_0, price_grid, test_lambdas)

  0.706334 seconds (368.86 k allocations: 15.653 MiB)


10-element Array{Float64,1}:
 0.311258
 0.873325
 1.34534 
 1.55611 
 1.61779 
 1.63168 
 1.63438 
 1.63488 
 1.63497 
 1.63498 

## Test `bellman_operator(Vguess, price_grid, lambda_simplex)` :

In [230]:
include("../julia/aguirregabiria_core.jl")
@time T_V, policy = bellman_operator(V_0, price_grid, simplex3d)

> Doing 1 of 6
4
> Doing 2 of 6
4
> Doing 3 of 6
5
> Doing 4 of 6
5
> Doing 5 of 6
5
> Doing 6 of 6
5
  1.286872 seconds (1.23 M allocations: 43.753 MiB)


(interpolate_V, [0.833333 3.37685e-315 3.37685e-315; 0.833333 3.37685e-315 3.37685e-315; … ; 0.944444 3.37685e-315 3.37685e-315; 0.944444 3.37685e-315 3.35042e-315])

In [231]:
T_V([0.4, 0.2, 0.4])

3.0416826021618695

In [233]:
for i in 1:size(simplex3d, 1)
    println(T_V(simplex3d[i,:]))
end

3.0607774882672927
3.0497592746690607
3.040085848338518
3.039933255786248
3.0388539687815435
3.0376220892245693


## Compute fixed point:

In [242]:
include("../julia/aguirregabiria_core.jl")
compute_fixed_point(V_0, price_grid, simplex3d)

Computed iterate 1 with error 8.4571
 !-- elapsed time: 1.048994361 seconds
Computed iterate 3 with error 6.6847
 !-- elapsed time: 0.525019983 seconds
Computed iterate 5 with error 5.3026
 !-- elapsed time: 0.509600013 seconds
Computed iterate 7 with error 4.2063
 !-- elapsed time: 0.540372443 seconds
Computed iterate 9 with error 3.3366
 !-- elapsed time: 0.47595541 seconds
Computed iterate 11 with error 2.6499
 !-- elapsed time: 0.450300828 seconds
Computed iterate 13 with error 2.1104
 !-- elapsed time: 0.879655534 seconds
Computed iterate 15 with error 1.6856
 !-- elapsed time: 0.485984252 seconds
Computed iterate 17 with error 1.3471
 !-- elapsed time: 0.525967833 seconds
Computed iterate 19 with error 1.0820
 !-- elapsed time: 0.508479877 seconds
Computed iterate 21 with error 0.8691
 !-- elapsed time: 0.529709542 seconds
Computed iterate 23 with error 0.6981
 !-- elapsed time: 0.466937069 seconds
Computed iterate 25 with error 0.5607
 !-- elapsed time: 0.407333816 seconds
Compu

(interpolate_V, [0.833333 3.37685e-315 3.37685e-315; 0.833333 3.37685e-315 3.37685e-315; … ; 0.944444 3.37685e-315 3.37685e-315; 0.944444 3.37685e-315 3.35042e-315], 0.04123928997986326)